In [23]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qtp
from IPython.display import display, Math, Latex

In [24]:
a = qtp.qutrit_basis()


In [25]:
print(a[0]*qtp.dag(a[0]))

Quantum object: dims = [[3], [3]], shape = [3, 3], type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


## Hamiltonian of two qutrits coupled directly

H^{direct} = \Sum_{j = A,B} [\omega_{j}\ket{1}\bra{1} +(2\omega_{j} - \Delta_{j})\ket{2}\bra{2}] + g[\ket{01}\bra{10} + \sqrt(2) \ket{02}{11} + 2\ket{12}\bra{21} + H.C.] 


In [27]:
def basis_kets():
    basis_set = qtp.qutrit_basis()
    ket0 = basis_set[0]
    ket1 = basis_set[1]
    ket2 = basis_set[2]
    bra0 = qtp.dag(ket0)
    bra1 = qtp.dag(ket1)
    bra2 = qtp.dag(ket2)
    return ket0,ket1,ket2,bra0,bra1,bra2
    

In [29]:
def qubit_term(freq1, anh1, freq2, anh2):
    ket0,ket1,ket2,bra0,bra1,bra2 = basis_kets()
    qubit_term1 = freq1*ket1*bra1 + (2*freq1 - anh1)*ket2*bra2
    qubit_term2 = freq2*ket1*bra1 + (2*freq2 - anh2)*ket2*bra2
    qubit_term = qtp.tensor(qubit_term1,qubit_term2)
    return qubit_term
    
    

In [30]:
def coupling_hamiltonian(coupling):
    ket0,ket1,ket2,bra0,bra1,bra2 = basis_kets()
    term1 = (qtp.tensor(ket0,ket1)*qtp.tensor(bra1,bra0) + np.sqrt(2)*qtp.tensor(ket0,ket2)*qtp.tensor(bra1,bra1)
             + np.sqrt(2)*qtp.tensor(ket2,ket0)*qtp.tensor(bra1,bra1) + 2* qtp.tensor(ket1,ket2)*qtp.tensor(bra2,bra1))
    term2 = qtp.dag(term1)
    
    return coupling*(term1 + term2)
    
    
    

In [33]:
def dir_coupled_qutrit(freq1, anh1, freq2, anh2, coupling):
    """
    freq 1 , 2 : splitting between bottom two levels (in GHZ)
    anh 1,2 : anharmonicity of two qubits in MHz
    g : coupling between qutrits 
    """
    qubit_term_hamiltonian = qubit_term(freq1, anh1, freq2, anh2)
    coupling_term = coupling_hamiltonian(coupling)
    hamiltonian_direct = qubit_term_hamiltonian + coupling_term
    return hamiltonian_direct
    
    
    
    
    

In [36]:
coupling = 0.3
dir_coupled_qutrit(5, .5, 5, .5, .3)

Quantum object: dims = [[3, 3], [3, 3]], shape = [9, 9], type = oper, isherm = True
Qobj data =
[[  0.           0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.3          0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.           0.42426407   0.
    0.           0.           0.        ]
 [  0.           0.3          0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.42426407   0.          25.           0.
    0.42426407   0.           0.        ]
 [  0.           0.           0.           0.           0.          47.5
    0.           0.6          0.        ]
 [  0.           0.           0.           0.           0.42426407   0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.           0.           0.6
    0.          47.5          0.

In [37]:
evolution_time = np.pi/(2*coupling)

In [45]:
U_evolution = (-1j * dir_coupled_qutrit(5, .5, 5, .5, .3) * evolution_time).expm()

In [46]:
U_evolution

Quantum object: dims = [[3, 3], [3, 3]], shape = [9, 9], type = oper, isherm = False
Qobj data =
[[ 1.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000-1.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.99845648+0.03785759j
   0.00000000+0.j         -0.00734631+0.01272417j  0.00000000+0.j
  -0.00154352+0.03785759j  0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000-1.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j         -0.00734631+0.012

In [ ]:
def qutrit_to_qubit(U_evolution):
    
    